In [17]:
import json
from pymongo import MongoClient
from pprint import pprint
from PIL import Image
import requests
import random
import time, os
from tqdm import tqdm
import io
import multiprocessing
from joblib import Parallel, delayed
from pyzipcode import ZipCodeDatabase
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import pickle
import numpy
import shutil

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import pairwise_distances

from keras import callbacks
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.layers import Input, Flatten, Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.applications.xception import Xception, preprocess_input, decode_predictions
from keras.models import load_model
m = load_model('room_classifier')

In [18]:
#need to repickle when classing is done
#EXTERIOR
arry_features_exterior = np.load('images_classed_arry/arry_features_exterior.npy')

pickle_in = open("images_classed_df/df_features_exterior.pickle","rb")
df_features_exterior = pickle.load(pickle_in)

#BEDROOM
arry_features_bedroom = np.load('images_classed_arry/arry_features_bedroom.npy')

pickle_in = open("images_classed_df/df_features_bedroom.pickle","rb")
df_features_bedroom = pickle.load(pickle_in)

#BATHROOM
arry_features_bathroom = np.load('images_classed_arry/arry_features_bathroom.npy')

pickle_in = open("images_classed_df/df_features_bathroom.pickle","rb")
df_features_bathroom = pickle.load(pickle_in)

#KITCHEN
arry_features_kitchen = np.load('images_classed_arry/arry_features_kitchen.npy')

pickle_in = open("images_classed_df/df_features_kitchen.pickle","rb")
df_features_kitchen = pickle.load(pickle_in)

#LIVING
arry_features_living = np.load('images_classed_arry/arry_features_living.npy')

pickle_in = open("images_classed_df/df_features_living.pickle","rb")
df_features_living = pickle.load(pickle_in)

In [19]:
client = MongoClient()
db = client.trulia_db
listings = db.listings.find_one({},{'_id':0, 'image_urls':0})

In [20]:
base_dir = 'input'
to_class = {0: 'bathroom',
 1: 'bedroom',
 2: 'exterior',
 3: 'kitchen',
 4: 'living',
 5: 'plan'}
IMG_WIDTH, IMG_HEIGHT = 299, 299 

datagen = ImageDataGenerator(rescale=1./255)

batch_size = 200

base_model = Xception(input_shape=(IMG_WIDTH, IMG_HEIGHT, 3), weights='imagenet', include_top=False)


# makes the prediction of the file path image passed as parameter 
def predict(file, model, to_class):
    im = load_img(file, target_size=(IMG_WIDTH, IMG_HEIGHT))
    x = img_to_array(im)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    index = model.predict(x).argmax()
    return to_class[index]

In [21]:
def class_input_df(input_fld):
    input_dir = os.listdir(input_fld)
    img_name_list = []
    img_predict_list = []
    
    for input_img in input_dir:
        img_name_list.append(input_img)
        img = input_fld+'/'+input_img
        img_predict_list.append(predict(img, m, to_class))
    comb_img = list(zip(img_name_list,img_predict_list))
    df = pd.DataFrame(comb_img, columns = ['img_name', 'img_class']) 
    return df


In [22]:
#appy this to all extract scripts
def extract_features(classes, sample_count):
    features = np.zeros(shape=(sample_count,10,10, 2048))
    generator = datagen.flow_from_directory(
        base_dir,
        target_size=(299, 299),
        shuffle=False,
        batch_size=batch_size,
        class_mode=None,
        classes=[classes])
    list_img = [img.split('/') for img in generator.filenames]
    img_df = pd.DataFrame(list_img, columns = ['class','img_name'])
    img_df['listingid'] = img_df['img_name'].apply(lambda x: x.split('_')[0])
    i = 0
    #pdb.set_trace()
    for inputs_batch in generator:
        features_batch = base_model.predict(inputs_batch)
        #pdb.set_trace()
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    features = numpy.array(features)
    features = np.reshape(features, (sample_count, 10 * 10 * 2048))
    return features


In [23]:
#moves inputs to right classes
def process_input_images(input_img_fld, output_img_fld):
    path = input_img_fld
    dest_path = output_img_fld
    for img in os.listdir(path):
        try:
            source_dir = path+"/"+str(img)
            room_class = predict(source_dir, m, to_class)
            dest_source = dest_path+"/"+room_class
            shutil.copy(src=source_dir,dst=dest_source)
        except:
            print('Failed: ' + source_dir)

In [24]:
#Text query
def get_close_zipcodes(zip_code,distance):
    zcdb = ZipCodeDatabase()
    in_radius = [z.zip for z in zcdb.get_zipcodes_around_radius(zip_code, distance)] # ('ZIP', radius in miles)
    return in_radius

def get_listings(zip_code, radius, min_price=0, max_price=999999999,property_type_in=None):
    if property_type_in is None:
        property_type = ['apartment','condo','coop','lot/land','mobile/manufactured','multi-family','single-family home','townhouse','unknown']
    else:
        property_type = [property_type_in]
    list_zip_codes = get_close_zipcodes(zip_code, radius)
    return db.listings.find({"$and":[
                                 {"zip_code" : {"$in" : list_zip_codes}},
                                 {"$expr": {"$gte": [ { "$toDouble": "$price" }, min_price ]}},
                                 {"$expr": {"$lte": [ { "$toDouble": "$price" }, max_price ]}},
                                 {"propertytype": {"$in" : property_type}}
                                ]}
                                 , {'_id':0,'image_urls':0})


In [25]:
#INSERT TEXT QUERY (zipcode, lower price, upper price, type of home ['apartment','condo','coop','lot/land','mobile/manufactured','multi-family','single-family home','townhouse','unknown'] )
results_json = get_listings(10314,4, 1.0,1000000.0)
query_results = pd.DataFrame(results_json)
query_results.drop_duplicates(subset='listingid', keep='first', inplace=True) 
query_result_listingid = set(query_results['listingid'])


In [38]:
query_results

,address,bath,bed,city,description,floorspace,listingid,locationlat,price,propertytype,state,url,zip_code
0,"564 Golfview Ct, Staten Island, NY 10314",2ba,2bd,Staten Island,"[joann dellarocca the listing agent is available to show this property at your convenience my cell is 718-614-2676 a must see! well maintained 2 bedroom duplex townhouse, walking distance to mall, x buses, excellent school district. enter into a large living room/dining room, eat in kitchen, bath, laundry room, 2nd level maste bedroom with sliders to terrace with park views, 2nd bedroom, full bath, attic with storage. great location. enjoy your summers at the pool and tennis courts.]","1,600 sqft",5068903154,40.5807,399000,townhouse,NY,https://www.trulia.com/p/ny/staten-island/564-golfview-ct-staten-island-ny-10314--2008862537,10314
1,"87 Pacific Ave, Staten Island, NY 10312",2ba,4bd,Staten Island,"[Colonial Style Home. This Home Features 4 Bedrooms, 2 Full Baths, Dining Room, (2) Eat In Kitchen & 1 Car Garage. Centrally Located To All. Don't Miss This Opportunity!]","2,517 sqft",1060324285,40.538177,669900,multi-family,NY,https://www.trulia.com/p/ny/staten-island/87-pacific-ave-staten-island-ny-10312--2008840702,10312
2,"146 Holland Ave, Staten Island, NY 10303",3ba,3bd,Staten Island,"[Fully detached one family located in the Mariners Harbor section of Staten Island. This house features living and dining combination, eat-in kitchen with access to rear deck and fenced yard, three bedrooms, master bedroom has walk-in closet, 2.5 baths, wood floors throughout, full basement for utilities, laundry and extra storage. Conveniently located minutes to shopping, schools and transportation. Employees or directors of JPMorgan Chase & Co. and its direct and indirect subsidiaries are strictly prohibited from directly or indirectly purchasing any property owned or serviced by or on behalf of JPMorgan Chase & Co. or its direct and indirect subsidiaries.]","1,204 sqft",1042302535,40.63646,294500,single-family home,NY,https://www.trulia.com/p/ny/staten-island/146-holland-ave-staten-island-ny-10303--2008738027,10303
3,"193 Benziger Ave, Staten Island, NY 10301",2ba,6bd,Staten Island,"[Vacant fully detached 2 story 2 family frame. 3 bedrooms over 3 bedrooms with a full finished basement. Property being sold in As-Is condition. No representations or warranties. Buyer responsible for any liens or violations. Buyer to pay NYC / NYS transfer taxes. Cash, rehab or FHA203k financing only.]","2,016 sqft",5068228540,40.639694,349900,multi-family,NY,https://www.trulia.com/p/ny/staten-island/193-benziger-ave-staten-island-ny-10301--2008723909,10301
4,"562 Midland Ave #C, Staten Island, NY 10306",1ba,2bd,Staten Island,"[Investment Property, fully renovated, Near V-Bridge. selling AS IS. Cash deal, because the house is waiting for the national grid to install the gas meter.]",450 sqft,1027334989,40.572407,198000,single-family home,NY,https://www.trulia.com/p/ny/staten-island/562-midland-ave-c-staten-island-ny-10306--2008784599,10306
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1266,"338 Crown Ave, Staten Island, NY 10312",3ba,4bd,Staten Island,"[2 FAMILY ALL BRICK HI-RANCH GRANITE COUNTER TOPS , NEW BATHROOMS, STUCO FRONT, PLUS LG 4 1/2 ROOM WALK IN APT AND FULL FINISHED BASEMENT AND MORE. Level 1: ENTRANCE FOYER, OFFICE,GARAGE,4 1/2 ROOM WALK IN APT Level 2: HUGE LIVIING ROOM/DINING ROOM COMBO , HUGE EIK, MBR,FULL BATH, BEDROOM, BEDROOM. Level 3: ATTIC FOR STORAGE. BASEMENT: 100% FINISHED HI-DRY HUGE FAMILY ROOM,FULL BATH LIVING ROOM.]","2,340 sqft",3153221056,40.54613,879000,single-family home,NY,https://www.trulia.com/p/ny/staten-island/338-crown-ave-staten-island-ny-10312--851281,10312
1267,"48 Oakville St, Staten Island, NY 10314",5ba,3bd,Staten Island,"[heart of willowbrook. massive 4 level, end unit attached. 8 rms, 5 baths. but not multi level - true colonial -all bedrooms (with 2 baths) upstairs from living room/dining room combo and spacious open eat-in kitchen wit

In [26]:
# pickle_out = open("list_input.pickle","wb")
# pickle.dump(query_result_listingid, pickle_out, protocol=2)
# pickle_out.close()

In [30]:
#step 1, upload pics and break out into groups
process_input_images('input/input/upload','input/input/classed_upload')
input_df = class_input_df('input/input/upload')
base_dir = 'input/input/classed_upload'
df_main = pd.DataFrame(data=query_result_listingid, columns=['listingid'])

In [34]:
#living
living_input_cnt = len(os.listdir(base_dir+"/living"))
if living_input_cnt >0: 
    input_living_features = extract_features('living', living_input_cnt)
    
    dist_living = pairwise_distances(arry_features_living,input_living_features,metric='cosine')
    df_dist_living = pd.DataFrame(data=dist_living)
    df_dist_living['avg'] = df_dist_living.mean(axis=1)
    df_dist_living_sum = df_dist_living.merge(df_features_living, how='inner', left_index=True, right_index=True, left_on=None, right_on=None)
    _ = df_dist_living_sum['listingid'].isin(query_results["listingid"])
    df = df_dist_living_sum[_].groupby(["listingid"]).apply(lambda x: x.sort_values(["avg"], ascending = False)).reset_index(drop=True)

    df2 = df.groupby('listingid').head(2)

    df_fin_living = df2.groupby(['listingid'])['avg'].mean().reset_index()
    df_fin_living.columns = ['listingid', 'living_avg']
    df_main = df_main.merge(df_fin_living, how='outer', left_on='listingid', right_on='listingid')
    
    
#kitchen
kitchen_input_cnt = len(os.listdir(base_dir+"/kitchen"))
if kitchen_input_cnt >0: 
    input_kitchen_features = extract_features('kitchen', kitchen_input_cnt)
    
    dist_kitchen = pairwise_distances(arry_features_kitchen,input_kitchen_features,metric='cosine')
    df_dist_kitchen = pd.DataFrame(data=dist_kitchen)
    df_dist_kitchen['avg'] = df_dist_kitchen.mean(axis=1)
    df_dist_kitchen_sum = df_dist_kitchen.merge(df_features_kitchen, how='inner', left_index=True, right_index=True, left_on=None, right_on=None)
    _ = df_dist_kitchen_sum['listingid'].isin(query_results["listingid"])
    df = df_dist_kitchen_sum[_].groupby(["listingid"]).apply(lambda x: x.sort_values(["avg"], ascending = False)).reset_index(drop=True)

    df2 = df.groupby('listingid').head(2)

    df_fin_kitchen = df2.groupby(['listingid'])['avg'].mean().reset_index()
    df_fin_kitchen.columns = ['listingid', 'kitchen_avg']

    df_main = df_main.merge(df_fin_kitchen, how='outer', left_on='listingid', right_on='listingid')


#exterior
exterior_input_cnt = len(os.listdir(base_dir+"/exterior"))
if exterior_input_cnt >0: 
    input_exterior_features = extract_features('exterior', exterior_input_cnt)
    
    dist_exterior = pairwise_distances(arry_features_exterior,input_exterior_features,metric='cosine')
    df_dist_exterior = pd.DataFrame(data=dist_exterior)
    df_dist_exterior['avg'] = df_dist_exterior.mean(axis=1)
    df_dist_exterior_sum = df_dist_exterior.merge(df_features_exterior, how='inner', left_index=True, right_index=True, left_on=None, right_on=None)
    _ = df_dist_exterior_sum['listingid'].isin(query_results["listingid"])
    df = df_dist_exterior_sum[_].groupby(["listingid"]).apply(lambda x: x.sort_values(["avg"], ascending = False)).reset_index(drop=True)

    df2 = df.groupby('listingid').head(2)

    df_fin_exterior = df2.groupby(['listingid'])['avg'].mean().reset_index()
    df_fin_exterior.columns = ['listingid', 'exterior_avg']

    df_main = df_main.merge(df_fin_exterior, how='outer', left_on='listingid', right_on='listingid')


#bedroom
bedroom_input_cnt = len(os.listdir(base_dir+"/bedroom"))
if bedroom_input_cnt >0: 
    input_bedroom_features = extract_features('bedroom', bedroom_input_cnt)
    
    dist_bedroom = pairwise_distances(arry_features_bedroom,input_bedroom_features,metric='cosine')
    df_dist_bedroom = pd.DataFrame(data=dist_bedroom)
    df_dist_bedroom['avg'] = df_dist_bedroom.mean(axis=1)
    df_dist_bedroom_sum = df_dist_bedroom.merge(df_features_bedroom, how='inner', left_index=True, right_index=True, left_on=None, right_on=None)
    _ = df_dist_bedroom_sum['listingid'].isin(query_results["listingid"])
    df = df_dist_bedroom_sum[_].groupby(["listingid"]).apply(lambda x: x.sort_values(["avg"], ascending = False)).reset_index(drop=True)

    df2 = df.groupby('listingid').head(2)

    df_fin_bedroom = df2.groupby(['listingid'])['avg'].mean().reset_index()
    df_fin_bedroom.columns = ['listingid', 'bedroom_avg']
    
    df_main = df_main.merge(df_fin_bedroom,how='outer', left_on='listingid', right_on='listingid')


#bathroom
bathroom_input_cnt = len(os.listdir(base_dir+"/bathroom"))
if bathroom_input_cnt >0: 
    input_bathroom_features = extract_features('bathroom', bathroom_input_cnt)
    
    dist_bathroom = pairwise_distances(arry_features_bathroom,input_bathroom_features,metric='cosine')
    df_dist_bathroom = pd.DataFrame(data=dist_bathroom)
    df_dist_bathroom['avg'] = df_dist_bathroom.mean(axis=1)
    df_dist_bathroom_sum = df_dist_bathroom.merge(df_features_bathroom, how='inner', left_index=True, right_index=True, left_on=None, right_on=None)
    _ = df_dist_bathroom_sum['listingid'].isin(query_results["listingid"])
    df = df_dist_bathroom_sum[_].groupby(["listingid"]).apply(lambda x: x.sort_values(["avg"], ascending = False)).reset_index(drop=True)

    df2 = df.groupby('listingid').head(2)

    df_fin_bathroom = df2.groupby(['listingid'])['avg'].mean().reset_index()
    df_fin_bathroom.columns = ['listingid', 'bathroom_avg']
    
    df_main = df_main.merge(df_fin_bathroom,how='outer', left_on='listingid', right_on='listingid')

#plan
plan_input_cnt = len(os.listdir(base_dir+"/plan"))
if plan_input_cnt >0: 
    input_plan_features = extract_features('plan', plan_input_cnt)
    
    dist_plan = pairwise_distances(arry_features_plan,input_plan_features,metric='cosine')
    df_dist_plan = pd.DataFrame(data=dist_plan)
    df_dist_plan['avg'] = df_dist_plan.mean(axis=1)
    df_dist_plan_sum = df_dist_plan.merge(df_features_plan, how='inner', left_index=True, right_index=True, left_on=None, right_on=None)
    _ = df_dist_plan_sum['listingid'].isin(query_results["listingid"])
    df = df_dist_plan_sum[_].groupby(["listingid"]).apply(lambda x: x.sort_values(["avg"], ascending = False)).reset_index(drop=True)

    df2 = df.groupby('listingid').head(2)

    df_fin_plan = df2.groupby(['listingid'])['avg'].mean().reset_index()
    df_fin_plan.columns = ['listingid', 'plan_avg']
    
    df_main = df_main.merge(df_fin_plan, how='outer', left_on='listingid', right_on='listingid')




Found 1 images belonging to 1 classes.
Found 1 images belonging to 1 classes.
Found 2 images belonging to 1 classes.


In [35]:
df_main['avg'] = df_main.mean(axis=1)
df_main = df_main.sort_values('avg').head(50)
df_main = df_main.merge(query_results, how='inner', left_index=True, right_index=True, left_on=None, right_on=None)

In [36]:
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [37]:
df_main

,listingid_x,living_avg_x,kitchen_avg_x,exterior_avg_x,living_avg_y,kitchen_avg_y,exterior_avg_y,avg,address,bath,...,city,description,floorspace,listingid_y,locationlat,price,propertytype,state,url,zip_code
742,3165258121,NaN,NaN,0.550279,NaN,NaN,0.550279,0.550279,"23 Fort Hill Cir, Staten Island, NY 10301",2ba,...,Staten Island,"[welcome to 23 fort hill circle! drop your bags and move right in to this spacious tudor, located on a enclave st george street. this home features; 3 bedrooms, 2 bathrooms, formal living room with gorgeous wood burning fireplace, formal dining room and family room, extra large eat-in kitchen, original wood floors with inlays & mouldings, master suite with walk in closet & sitting room/office, built in garage & full finished basement. 1 mile to si ferry terminal & short distance empire outlets.]","1,944 sqft",5049139653,40.643463,699000,single-family home,NY,https://www.trulia.com/p/ny/staten-island/23-fort-hill-cir-staten-island-ny-10301--2008724942,10301
1046,5067910166,NaN,NaN,0.559877,NaN,NaN,0.559877,0.559877,"198 Carlyle Grn, Staten Island, NY 10312",2ba,...,Staten Island,[Beautiful newly renovated fully detached single family house with hardwood floors thru-out. Oversized sliding glass door to trex deck back yard and windows bring lots of natural sun light into the formal living and dining rooms. Large four bedrooms and two full bathrooms. Huge open finished basement with lots of storage. New siding.Convenient to all.Don't miss out!!],"1,344 sqft",5045252772,40.555347,585000,single-family home,NY,https://www.trulia.com/p/ny/staten-island/198-carlyle-grn-staten-island-ny-10312--2008826272,10312
526,3011275117,NaN,NaN,0.562847,NaN,NaN,0.562847,0.562847,"18 Caroline St, Staten Island, NY 10310",4ba,...,Staten Island,"[2 Family Semi-attached Townhome, 4 total bathrooms, hardwood floors throughout, large master bedroom with his/her closets. 3 Car driveway. Side entrance 1bedroom apartment with separate utilities. Full 800 sqft unfinished basement with separate entrance. 10 mins from Staten Island Ferry. Close to transportation, restaurants, shopping, banks etc.]","1,350 sqft",5056163532,40.63378,491500,townhouse,NY,https://www.trulia.com/p/ny/staten-island/18-caroline-st-staten-island-ny-10310--2008815806,10310
578,1063970400,NaN,NaN,0.566992,NaN,NaN,0.566992,0.566992,"478 Amherst Ave, Staten Island, NY 10306",4ba,...,Staten Island,"[19524h-new development - 13 new houses in a superb location! gorgeous large 3200 sq ft houses offer 4 bedrooms, 4 bath, wide open floor plan, custom eik, 3 zoned heating, 2 zone a/c, huge balcony large family room w/3/4 bath and a lot more. model home located at 470 amherst ave. level 1: family room, 3/4 bath, sliders to deck, garage. level 2: huge living room, dinette, eik, balcony, 1/2 bath. level 3: master suite w/separate bath & walk in closet, 3 bedrooms, 4 piece bath.]","3,200 sqft",5063716500,40.557884,839000,single-family home,NY,https://www.trulia.com/p/ny/staten-island/478-amherst-ave-staten-island-ny-10306--2373596169,10306
476,5066365321,NaN,0.581903,0.576879,NaN,0.542193,0.576879,0.569464,"57 Lyceum Ct, Staten Island, NY 10310",3ba,...,Staten Island,"[Perfect starter home! Lovely and newly updated 3 bedroom single family home located on a beautiful neighborhood of Staten Island. Features 3 bedrooms, 2.5 baths, hardwood floors throughout, lots of windows, backyard, finished basement, private driveway and solar panels with another year for a tax rebate. Short trip to Brooklyn. Property Layout: BASEMENT: Finished basement, includes LED lights which can be controlled by phone; laundry room with occupancy sensor to control light. FIRST FLOOR: Kitchen with hardwood cabinetry, ample and sunny living room with LED lights throughout and hardwood floors. SECOND FLOOR: Includes 3 bedrooms with closets, full bathroom and hardwood floors]","1,176 sqft",5061774503,40.630474,445000,single-family home,NY,https://www.trulia.com/p/ny/staten-island/57-lyceum-ct-staten-island